In [23]:
#imports
import numpy as np
import tensorflow.compat.v1 as tf
tf.compat.v1.disable_eager_execution()
tf.disable_v2_behavior()
import tensorflow.compat.v1.keras as K
import matplotlib.pyplot as plt

In [24]:
# Task 1. Inception Block
"""
Write a function that builds an inception block as described in Going Deeper
with Convolutions (2014)
"""
def inception_block(A_prev, filters):
    """
    A_prev: output from the previous layer
    filters: tuple containing F1, F3R, F3, F5R, F5, FPP
        F1: number of filters in the 1x1 convolution
        F3R: number of filters in the 1x1 convolution before the 3x3
        convolution
        F3: number of filters in the 3x3 convolution
        F5R: number of filters in the 1x1 convolution before the 5x5
        convolution
        F5: number of filters in the 5x5 convolution
        FPP: number of filters in the 1x1 convolution after the max pooling
    All convolutions inside the inception block should use a ReLU activation
    Returns: the concatenated output of the inception block
    """
    F1 = filters[0]
    F3R = filters[1]
    F3 = filters[2]
    F5R = filters[3]
    F5 = filters[4]
    FPP = filters[5]

    conv_1x1 = K.layers.Conv2D(filters=F1,
                               kernel_size=(1, 1),
                               padding='same',
                               activation='relu'
                               )(A_prev)

    conv_1x1_3x3 = K.layers.Conv2D(filters=F3R,
                                   kernel_size=(1, 1),
                                   padding='same',
                                   activation='relu'
                                   )(A_prev)

    conv_3x3 = K.layers.Conv2D(filters=F3,
                               kernel_size=(3, 3),
                               padding='same',
                               activation='relu'
                               )(conv_1x1_3x3)

    conv_1x1_5x5 = K.layers.Conv2D(filters=F5R,
                                   kernel_size=(1, 1),
                                   padding='same',
                                   activation='relu'
                                   )(A_prev)

    conv_5x5 = K.layers.Conv2D(filters=F5,
                               kernel_size=(5, 5),
                               padding='same',
                               activation='relu'
                               )(conv_1x1_5x5)

    max_pool_3x3 = K.layers.MaxPooling2D(pool_size=(3, 3),
                                         strides=1,
                                         padding='same'
                                         )(A_prev)

    conv_1x1_pooled = K.layers.Conv2D(filters=FPP,
                                      kernel_size=(1, 1),
                                      padding='same',
                                      activation='relu'
                                      )(max_pool_3x3)

    output = K.layers.Concatenate()([
        conv_1x1, conv_3x3, conv_5x5, conv_1x1_pooled
    ])

    return output


In [25]:
# 0-main
X = K.Input(shape=(224, 224, 3))
Y = inception_block(X, [64, 96, 128, 16, 32, 32])
model = K.models.Model(inputs=X, outputs=Y)
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_145 (Conv2D)            (None, 224, 224, 96  384         ['input_10[0][0]']               
                                )                                                                 
                                                                                                  
 conv2d_147 (Conv2D)            (None, 224, 224, 16  64          ['input_10[0][0]']               
                                )                                                           

In [26]:
# Task 1. Inception Network
"""
Write a function that builds the inception network as described in
    `Going Deeper with Convolutions (2014)`
"""
def inception_network():
    """
    Assume input data will have shape (224, 224, 3)
    All convolutions inside and outside the inception block should use a ReLU
    activation
    Returns the Keras model
    """
    inputs = K.Input(shape=(224, 224, 3))

    conv_7x7_2s = K.layers.Conv2D(filters=64,
                                 kernel_size=(7, 7),
                                 strides=2,
                                 padding='same',
                                 activation='relu'
                                 )(inputs)

    MaxPool3x3_2s = K.layers.MaxPooling2D(pool_size=(3, 3),
                                          strides=2,
                                          padding='same'
                                          )(conv_7x7_2s)

    conv_1x1_1v = K.layers.Conv2D(filters=64,
                                  kernel_size=(1, 1),
                                  padding='valid',
                                  activation='relu'
                                  )(MaxPool3x3_2s)

    conv_3x3_1s = K.layers.Conv2D(filters=192,
                                  kernel_size=(3, 3),
                                  strides=1,
                                  padding='same',
                                  activation='relu'
                                  )(conv_1x1_1v)

    MaxPool3x3_2s_1 = K.layers.MaxPooling2D(pool_size=(3, 3),
                                            strides=2,
                                            padding='same'
                                            )(conv_3x3_1s)

    inception_layer_0 = inception_block(MaxPool3x3_2s_1,
                                        [64, 96, 128, 16, 32, 32])

    inception_layer_1 = inception_block(inception_layer_0,
                                        [128, 128, 192, 32, 96, 64])

    MaxPool3x3_2s_2 = K.layers.MaxPooling2D(pool_size=(3, 3),
                                            strides=2,
                                            padding='same'
                                            )(inception_layer_1)

    inception_layer_2 = inception_block(MaxPool3x3_2s_2,
                                        [192, 96, 208, 16, 48, 64])

    inception_layer_3 = inception_block(inception_layer_2,
                                        [160, 112, 224, 24, 64, 64])

    inception_layer_4 = inception_block(inception_layer_3,
                                        [128, 128, 256, 24, 64, 64])

    inception_layer_5 = inception_block(inception_layer_4,
                                        [112, 144, 288, 32, 64, 64])

    inception_layer_6 = inception_block(inception_layer_5,
                                        [256, 160, 320, 32, 128, 128])

    MaxPool3x3_2s_3 = K.layers.MaxPooling2D(pool_size=(3, 3),
                                            strides=2,
                                            padding='same'
                                            )(inception_layer_6)

    inception_layer_7 = inception_block(MaxPool3x3_2s_3,
                                        [256, 160, 320, 32, 128, 128])

    inception_layer_8 = inception_block(inception_layer_7,
                                        [384, 192, 384, 48, 128, 128])

    AvgPool7x7_1v = K.layers.AveragePooling2D(pool_size=(7, 7),
                                              strides=1,
                                              padding='valid'
                                              )(inception_layer_8)

    dropout = K.layers.Dropout(.4)(AvgPool7x7_1v)

    softmax = K.layers.Dense(units=1000,
                             activation='softmax'
                             )(dropout)

    return K.Model(inputs=inputs, outputs=softmax)

In [27]:
# 1-main
model = inception_network()
model.summary()

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_150 (Conv2D)            (None, 112, 112, 64  9472        ['input_11[0][0]']               
                                )                                                                 
                                                                                                  
 max_pooling2d_29 (MaxPooling2D  (None, 56, 56, 64)  0           ['conv2d_150[0][0]']             
 )                                                                                          

In [28]:
# Task 2. Identity Block
"""
Write a function that builds an identity block as described in
    `Deep Residual Learning for Image Recognition(2015)`
"""
def identity_block(A_prev, filters):
    """
    A_prev: output from the previous layer
    filters: tuple or list containing F11, F3, F12 respectively
        F11: number of filters in the first 1x1 convolution
        F3: number of filters in the 3x3 convolution
        F12: numnber of filters in the second 1x1 convolution
    All convolutions inside the block should be followed by batch normalization
        along the channels axis and a rectified linear activation (ReLU)
    All weights should use he normal initialization
    Returns the activated output of the identity block
    """
    F11 = filters[0]
    F3 = filters[1]
    F12 = filters[2]

    init = K.initializers.he_normal()

    conv1x1_0 = K.layers.Conv2D(filters=F11,
                                kernel_size=(1, 1),
                                strides=1,
                                padding='same',
                                kernel_initializer=init
                                )(A_prev)

    batch_norm_0 = K.layers.BatchNormalization(axis=3)(conv1x1_0)

    relu_0 = K.layers.ReLU()(batch_norm_0)

    conv3x3_1 = K.layers.Conv2D(filters=F3,
                                kernel_size=(3, 3),
                                strides=1,
                                padding='same',
                                kernel_initializer=init)(relu_0)

    batch_norm_1 = K.layers.BatchNormalization(axis=3)(conv3x3_1)

    relu_1 = K.layers.ReLU()(batch_norm_1)

    conv1x1_2 = K.layers.Conv2D(filters=F12,
                                kernel_size=(1, 1),
                                strides=1,
                                padding='same',
                                kernel_initializer=init)(relu_1)

    batch_norm_2 = K.layers.BatchNormalization(axis=3)(conv1x1_2)

    add_layer = K.layers.Add()([batch_norm_2, A_prev])

    relu_output = K.layers.ReLU()(add_layer)

    return relu_output


In [29]:
# 2-main
X = K.Input(shape=(224, 224, 256))
Y = identity_block(X, [64, 64, 256])
model = K.models.Model(inputs=X, outputs=Y)
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_12 (InputLayer)          [(None, 224, 224, 2  0           []                               
                                56)]                                                              
                                                                                                  
 conv2d_207 (Conv2D)            (None, 224, 224, 64  16448       ['input_12[0][0]']               
                                )                                                                 
                                                                                                  
 batch_normalization_18 (BatchN  (None, 224, 224, 64  256        ['conv2d_207[0][0]']             
 ormalization)                  )                                                           

In [30]:
# Task 3. Projection Block
"""
Write a function that builds an projection block as described in
    `Deep Residual Learning for Image Recognition(2015)`
"""
def projection_block(A_prev, filters, s=2):
    """
    A_prev: output from the previous layer
    filters: tuple or list containing F11, F3, F12 respectively
        F11: number of filters in the first 1x1 convolution
        F3: number of filters in the 3x3 convolution
        F12: numnber of filters in the second 1x1 convolution
    s: stride for the first convolution in both the main path and the shortcut
        connection
    All convolutions inside the block should be followed by batch normalization
        along the channels axis and a rectified linear activation (ReLU)
    All weights should use he normal initialization
    Returns the activated output of the identity block
    """
    F11 = filters[0]
    F3 = filters[1]
    F12 = filters[2]

    init = K.initializers.he_normal()

    conv1x1_0 = K.layers.Conv2D(filters=F11,
                                kernel_size=(1, 1),
                                strides=s,
                                padding='same',
                                kernel_initializer=init
                                )(A_prev)

    batch_norm_0 = K.layers.BatchNormalization(axis=3)(conv1x1_0)

    relu_0 = K.layers.ReLU()(batch_norm_0)

    conv3x3_1 = K.layers.Conv2D(filters=F3,
                                kernel_size=(3, 3),
                                strides=1,
                                padding='same',
                                kernel_initializer=init)(relu_0)

    batch_norm_1 = K.layers.BatchNormalization(axis=3)(conv3x3_1)

    relu_1 = K.layers.ReLU()(batch_norm_1)

    conv1x1_2 = K.layers.Conv2D(filters=F12,
                                kernel_size=(1, 1),
                                strides=1,
                                padding='same',
                                kernel_initializer=init)(relu_1)

    batch_norm_2 = K.layers.BatchNormalization(axis=3)(conv1x1_2)

    conv1x1_shortcut = K.layers.Conv2D(filters=F12,
                                       kernel_size=(1, 1),
                                       strides=s,
                                       padding='same',
                                       kernel_initializer=init)(A_prev)

    batch_norm_shortcut = K.layers.BatchNormalization(axis=3)(conv1x1_shortcut)

    add_layer = K.layers.Add()([batch_norm_2, batch_norm_shortcut])

    relu_output = K.layers.ReLU()(add_layer)

    return relu_output

In [31]:
# 3-main
X = K.Input(shape=(224, 224, 3))
Y = projection_block(X, [64, 64, 256])
model = K.models.Model(inputs=X, outputs=Y)
model.summary()

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_13 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_210 (Conv2D)            (None, 112, 112, 64  256         ['input_13[0][0]']               
                                )                                                                 
                                                                                                  
 batch_normalization_21 (BatchN  (None, 112, 112, 64  256        ['conv2d_210[0][0]']             
 ormalization)                  )                                                          